In [234]:
import pandas as pd
import sys, os
import tensorflow as tf
import shape_helper
from tensorflow import keras
from tensorflow.keras import layers

In [238]:

#the following bits are basically just to deal with translations and rotations in a pretty efficient manner. Note the 
#translation+removal functions have no for loops so it's pretty quick 
#Let's make the goal shape into a bunch of 1's and 0's representing where we want pieces
def mattoones(goalshape1):
    for i in range(len(goalshape1)):
        for k in range(len(np.transpose(goalshape1))):
            if goalshape1[i][k]!='0.0' and goalshape1[i][k]!=0:
                goalshape1[i][k]=1
    return goalshape1
#Remove all rows/columns at edges with only 0's in order to standardize calculations 
def translator(matrix):
    sums=0;
    while(sum(matrix[0])==0):
        if (sum(matrix[0]==0)):
            matrix=np.delete(matrix,0,0)

    i=len(matrix)-1
    while(sums==0):
        sums=sum(matrix[i]);
        if(sums==0):
            matrix=np.delete(matrix,i,0)
        i=i-1
    return(matrix)

#(*use the above function but for columns too*)
def removal(matrixer):
    c=translator(matrixer);
    c=translator(np.transpose(c))
    return(np.transpose(c))
def sqdiff(m1,m2):
    return (sum(sum((m1-m2)*(m1-m2))))
#I think this lets us visualize chains as 1/0s instead of HP but it's not very useful 
def graphicchainno(chain):
    boundary=np.floor(np.sqrt(len(chain))*2);
    lattice=np.ndarray.tolist(np.zeros([int(boundary),int(boundary)]))
    for i in range(len(chain)):
        lattice[chain[i][0]][chain[i][1]]=1
    lattice=np.array(lattice)
    return(lattice)

In [239]:
x=1
model=keras.Sequential([
    #Entry block: create four masks, and run
    layers.Conv2D(filters=4,kernel_size=3,activation="relu",padding='same',input_shape=[15,15,1]),
    layers.MaxPool2D(),
    #Block 2
    layers.Conv2D(filters=16, kernel_size=3,activation="relu",padding='same'),
    layers.MaxPool2D(),

    #Third hidden layer
    layers.Conv2D(filters=128,kernel_size=3,activation="relu",padding='same'),
    #output 
    layers.Flatten(),
    layers.Dense(units=6,activation="relu"),
    layers.Dense(units=x,activation="sigmoid")
])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 15, 15, 4)         40        
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 7, 7, 4)          0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 7, 7, 16)          592       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 3, 3, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 3, 3, 128)         18560     
                                                                 
 flatten_7 (Flatten)         (None, 1152)             

1+1

In [243]:
#prepare the data 
import os,warnings
import matplotlib as plt
import matplotlib.pyplot as publish_display_data
from matplotlib import gridspec
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells
def deserialize_shape(string,n):
    """
    Takes a string and decodes it into a matrix
    """
    matrix = np.array([])

    def decode_count(char):
        if char.isdigit():
            return int(char)
        return ord(char)-87
    
    for i in range(0,len(string), 2):
        matrix = np.append(matrix, np.full(decode_count(string[i]), int(string[i+1])))
    return np.asarray(matrix).reshape(n, n)

In [250]:
#set datasets 
#from sklearn.model_selection import train_test_split
import random
import shape_helper
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/Sequences_rows (2).csv')
targets=df['length']
startshape=df['shape_mapping']
validtargets=[]
validshapes=[]
arraysize=12
#create our array entries 
for i in range(len(startshape)):
    np.pad(c,[[(10-x),0],[10-y,0]])
    startshape[i]=removal(np.asarray(shape_helper.deserialize_shape(startshape[i]),np.float32))
    x,y=np.shape(startshape[i])
    startshape[i]=np.pad(startshape[i],[[(10-x),0],[10-y,0]])
#create our initial placement entries 


for i in range(len(startshape)):
    prob=random.random()
    if prob<.1:
        validshapes.append(startshape[i])
        validtargets.append(targets[i])
#startshapes=tf.stack(startshape,np.float32)
        






In [260]:
df['path'][1]

'0,0 0,1 -1,1 -1,0 -2,0 -3,0 -3,-1 -3,-2 -2,-2 -1,-2 -1,-3 0,-3 0,-2 0,-1 -1,-1 -2,-1 '

In [230]:
x,y=np.shape(removal(validshapes[1]))


In [249]:
np.pad(c,[[(10-x),0],[10-y,0]])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1.]], dtype=float32)

In [182]:
import math as math
def path_to_shape(path,grid_sizes):
    grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # actual grid for mapping
    temp_grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # temp grid to hold the array before alignment
    for coord in path:
        temp_grid[coord[1]+round(grid_sizes/2)][coord[0]+ round(grid_sizes/2)] = 1 # path transferred onto grid but uncentered
    temp_grid[round(grid_sizes/2)][round(grid_sizes/2)] = 2  
    temp_grid = np.pad(temp_grid, 1) # zero padding to avoid multiplying by 0 when calculating moments

    # find centroid of temp_grid
    m_00 = len(path) # non-zero residues
    m_01 = 0
    for row_n, row in enumerate(temp_grid):
        if np.any(row!=0):
            m_01 += row_n * np.count_nonzero(row)
    m_10 = 0
    for col_n in range(np.shape(temp_grid)[1]):
        if np.any(temp_grid[:, col_n]!=0):
            m_10 += col_n *  np.count_nonzero(temp_grid[:, col_n])

    # coordinates of centroid
    n_centroid = math.floor((m_10/m_00))
    m_centroid = math.floor((m_01/m_00))
    centroid = (m_centroid, n_centroid)
    #print("centroid ==>", centroid)
    # align temp_grid onto grid
    dev_m = round(grid_sizes/2)-centroid[0]
    dev_n = round(grid_sizes/2)-centroid[1]
    coord_list = np.nonzero(temp_grid)
    print("startpos ==>", (dev_m,dev_n))
    start= (round(grid_sizes/2)+dev_m,round(grid_sizes/2)+dev_n)
    for i in range(len(coord_list[0])):
        grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 1
        #if i==0:
            #grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 2
    
    return temp_grid, grid,start
def start_to_class(coords,grid_size):
    return coords[0]*grid_size+coords[1]

In [183]:

#Obtain a list of start positions in relation to centroids and shapes 
c=path_to_shape([[0,0],[0,1], [-1,1],[ -2,1], [-2,0],[ -1,0], [-1,-1],[ 0,-1],[0,-2],[ 1,-2], [2,-2],[ 2,-1], [2,0], [1,0], [1,-1]],10)
k=start_to_class(c[2],10)
tk=tf.convert_to_tensor(k)



startpos ==> (0, -1)


In [184]:
df

,sequence_id,sequence,degeneracy,length,energy,shape_mapping,path
0,-6506029740256395023,HPHPPPPHPHPHPHH,2,15,-6,Ə031m041l041m021n021ĩ0,"0,0 0,1 -1,1 -1,2 -1,3 -2,3 -2,2 -2,1 -3,1 -3,..."
1,6182686246958916932,HPHPPHPHPHPHPHH,2,15,-7,Ɛ031l041k051k031Ń0,"0,0 0,1 -1,1 -2,1 -2,0 -1,0 -1,-1 0,-1 0,-2 1,..."
2,-6290340639949105171,HPPHHHPHHPHPPHP,2,15,-6,Ɛ021m041l041m031n021Ĩ0,"0,0 0,1 -1,1 -1,0 -1,-1 0,-1 0,-2 -1,-2 -1,-3 ..."
3,-19529217640658864,HPHPPPPHPHPPPHH,2,15,-5,Ɛ031l041l041m021n021ĩ0,"0,0 0,1 1,1 1,2 1,3 2,3 2,2 2,1 3,1 3,0 3,-1 2..."
4,-7378937463594783569,HPHPPHPPPHPPHHP,2,15,-5,Ə031l051k051m021ł0,"0,0 0,1 -1,1 -1,2 -2,2 -2,1 -3,1 -4,1 -4,0 -3,..."
...,...,...,...,...,...,...,...
95,7967242413348939084,HPHPPPHPPPPHPHP,2,15,-4,Ÿ011n031k051k041n021ł0,"0,0 0,1 -1,1 -1,2 -1,3 -2,3 -2,2 -3,2 -4,2 -4,..."
96,-5155282045392613402,HPHPPPHPPPPHHHP,2,15,-4,Ÿ011n031k051k041n021ł0,"0,0 0,1 -1,1 -1,2 -1,3 -2,3 -2,2 -3,2 -4,2 -4,..."
97,-4092807018667426668,HPHPPPHPPPPHHHP,2,15,-4,ŷ011n031m051l041l021ł0,"0,0 0,1 1,1 1,2 1,3 2,3 2,2 3,2 4,2 4,1 3,1 2,..."
98,2361056144606801406,HPHPPPHPPPHHHHP,2,15,-4,Ÿ011n031k051k041n021ł0,"0,0 0,1 -1,1 -1,2 -1,3 -2,3 -2,2 -3,2 -4,2 -4,..."


In [95]:
model.predict(
startshape
)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
batch_size = 32
img_height = 25
img_width = 25
train_ds=tf.keras.utils.image_dataset_from_directory()
